# Introduction to Vector

Vector is a Python 3.8+ library (Python 3.6 and 3.7 supported till `v0.9.0` and `v1.0.0`, respectively) for 2D, 3D, and [Lorentz vectors](https://en.wikipedia.org/wiki/Special_relativity#Physics_in_spacetime), especially _arrays of vectors_, to solve common physics problems in a NumPy-like way.

Main features of Vector:

   * Pure Python with NumPy as its only dependency. This makes it easier to install.
   * Vectors may be represented in a variety of coordinate systems: Cartesian, cylindrical, spherical, and any combination of these with time or proper time for Lorentz vectors. In all, there are 12 coordinate systems: {$x$-$y$ vs $\rho$-$\phi$ in the azimuthal plane} × {$z$ vs $\theta$ vs $\eta$ longitudinally} × {$t$ vs $\tau$ temporally}.
   * Uses names and conventions set by [ROOT](https://root.cern/)'s [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html) and [Math::LorentzVector](https://root.cern.ch/doc/master/classROOT_1_1Math_1_1LorentzVector.html), as well as [scikit-hep/math](https://github.com/scikit-hep/scikit-hep/tree/master/skhep/math), [uproot-methods TLorentzVector](https://github.com/scikit-hep/uproot3-methods/blob/master/uproot3_methods/classes/TLorentzVector.py), [henryiii/hepvector](https://github.com/henryiii/hepvector), and [coffea.nanoevents.methods.vector](https://coffeateam.github.io/coffea/modules/coffea.nanoevents.methods.vector.html).
   * Implemented on a variety of backends:
      - pure Python objects
      - NumPy arrays of vectors (as a [structured array](https://numpy.org/doc/stable/user/basics.rec.html) subclass)
      - [Awkward Arrays](https://awkward-array.org/) of vectors
      - potential for more: CuPy, TensorFlow, Torch, JAX...
   * Implemented in [Numba](https://numba.pydata.org/) for JIT-compiled calculations on vectors.
   * Distinction between geometrical vectors, which have a minimum of attribute and method names, and vectors representing momentum, which have synonyms like `pt` = `rho`, `energy` = `t`, `mass` = `tau`.

This notebook requires Vector, NumPy, Awkward Array, and Numba to run.

In [1]:
import awkward as ak  # at least version 1.2.0rc5
import numba as nb
import numpy as np

import vector

## Constructing a vector or an array of vectors

The easiest way to create one or many vectors is with a helper function:

   * `vector.obj` to make a pure Python vector object,
   * `vector.arr` to make a NumPy array of vectors (lowercase, like `np.array`),
   * `vector.awk` to make an Awkward Array of vectors (uppercase, like `ak.Array`).
   * `vector.zip` to make an Awkward Array of vectors (similar to `ak.zip`)

### Pure Python vectors

You can directly use the `VectorObject` classes to construct object type vectors:

In [2]:
vector.VectorObject2D(x=1.1, y=2.2)

VectorObject2D(x=1.1, y=2.2)

In [3]:
vector.MomentumObject3D(px=1.1, py=2.2, pz=3.3)

MomentumObject3D(px=1.1, py=2.2, pz=3.3)

In [4]:
vector.VectorObject4D(x=1.1, y=2.2, eta=3.3, tau=4.4)

VectorObject4D(x=1.1, y=2.2, eta=3.3, tau=4.4)

and so on for every class.

Or, you can use a single wrapper function to construct all possible combinations of
object type vectors:

In [5]:
vector.obj(x=3, y=4)  # Cartesian 2D vector

VectorObject2D(x=3, y=4)

In [6]:
vector.obj(rho=5, phi=0.9273)  # same in polar coordinates

VectorObject2D(rho=5, phi=0.9273)

In [7]:
vector.obj(x=3, y=4).isclose(
    vector.obj(rho=5, phi=0.9273)
)  # use "isclose" unless they are exactly equal

True

In [8]:
vector.obj(x=3, y=4, z=-2)  # Cartesian 3D vector

VectorObject3D(x=3, y=4, z=-2)

In [9]:
vector.obj(x=3, y=4, z=-2, t=10)  # Cartesian 4D vector

VectorObject4D(x=3, y=4, z=-2, t=10)

In [10]:
vector.obj(
    rho=5, phi=0.9273, eta=-0.39, t=10
)  # in rho-phi-eta-t cylindrical coordinates

VectorObject4D(rho=5, phi=0.9273, eta=-0.39, t=10)

In [11]:
vector.obj(
    pt=5, phi=0.9273, eta=-0.39, E=10
)  # use momentum-synonyms to get a momentum vector

MomentumObject4D(pt=5, phi=0.9273, eta=-0.39, E=10)

In [12]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10) == vector.obj(
    pt=5, phi=0.9273, eta=-0.390035, E=10
)

False

In [13]:
vector.obj(
    rho=5, phi=0.9273, eta=-0.39, t=10
).tau  # geometrical vectors have to use geometrical names ("tau", not "mass")

8.426194916448265

In [14]:
vector.obj(
    pt=5, phi=0.9273, eta=-0.39, E=10
).mass  # momentum vectors can use momentum names (as well as geometrical ones)

8.426194916448265

In [15]:
vector.obj(
    pt=5, phi=0.9273, theta=1.9513, mass=8.4262
)  # any combination of azimuthal, longitudinal, and temporal coordinates is allowed

MomentumObject4D(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)

In [16]:
vector.obj(x=3, y=4, z=-2, t=10).isclose(
    vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)
)

True

In [17]:
# Test instance type for any level of granularity.
(
    isinstance(
        vector.obj(x=1.1, y=2.2), vector.Vector
    ),  # is a vector or array of vectors
    isinstance(vector.obj(x=1.1, y=2.2), vector.Vector2D),  # is 2D (not 3D or 4D)
    isinstance(
        vector.obj(x=1.1, y=2.2), vector.VectorObject
    ),  # is a vector object (not an array)
    isinstance(vector.obj(px=1.1, py=2.2), vector.Momentum),  # has momentum synonyms
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Planar
    ),  # has transverse plane (2D, 3D, or 4D)
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Spatial
    ),  # has all spatial coordinates (3D or 4D)
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Lorentz
    ),  # has temporal coordinates (4D)
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).azimuthal, vector.AzimuthalXY
    ),  # azimuthal coordinate type
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).longitudinal, vector.LongitudinalZ
    ),  # longitudinal coordinate type
    isinstance(
        vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).temporal, vector.TemporalT
    ),  # temporal coordinate type
)

(True, True, True, True, True, True, True, True, True, True)

The allowed keyword arguments for 2D vectors are:

   * `x` and `y` for Cartesian azimuthal coordinates,
   * `px` and `py` for momentum,
   * `rho` and `phi` for polar azimuthal coordinates,
   * `pt` and `phi` for momentum.

For 3D vectors, you need the above and:

   * `z` for the Cartesian longitudinal coordinate,
   * `pz` for momentum,
   * `theta` for the spherical polar angle (from $0$ to $\pi$, inclusive),
   * `eta` for pseudorapidity, which is a kind of spherical polar angle.

For 4D vectors, you need the above and:

   * `t` for the Cartesian temporal coordinate,
   * `E` or `energy` to get four-momentum,
   * `tau` for the "proper time" (temporal coordinate in the vector's rest coordinate system),
   * `M` or `mass` to get four-momentum.

Since momentum vectors have momentum-synonyms _in addition_ to the geometrical names, any momentum-synonym will make the whole vector a momentum vector.

If you want to bypass the dimension and coordinate system inference through keyword arguments (e.g. for static typing), you can use specialized constructors:

In [18]:
vector.VectorObject2D.from_xy(1.1, 2.2)

VectorObject2D(x=1.1, y=2.2)

In [19]:
vector.MomentumObject3D.from_rhophiz(1.1, 2.2, 3.3)

MomentumObject3D(pt=1.1, phi=2.2, pz=3.3)

In [20]:
vector.VectorObject4D.from_xyetatau(1.1, 2.2, 3.3, 4.4)

VectorObject4D(x=1.1, y=2.2, eta=3.3, tau=4.4)

and so on, for all combinations of azimuthal, longitudinal, and temporal coordinates, geometric and momentum-flavored.

### NumPy arrays of vectors

You can directly use the `VectorNumpy` classes to construct object type vectors:

In [21]:
# NumPy-like arguments (literally passed through to NumPy)
vector.VectorNumpy2D(
    [(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
    dtype=[("x", float), ("y", float)],
)

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [22]:
# Pandas-like arguments (dict from names to column arrays)
vector.VectorNumpy2D({"x": [1.1, 1.2, 1.3, 1.4, 1.5], "y": [2.1, 2.2, 2.3, 2.4, 2.5]})

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [23]:
# As with objects, the coordinate system and dimension is taken from the names of the fields.
vector.VectorNumpy4D(
    {
        "x": [1.1, 1.2, 1.3, 1.4, 1.5],
        "y": [2.1, 2.2, 2.3, 2.4, 2.5],
        "z": [3.1, 3.2, 3.3, 3.4, 3.5],
        "t": [4.1, 4.2, 4.3, 4.4, 4.5],
    }
)

VectorNumpy4D([(1.1, 2.1, 3.1, 4.1), (1.2, 2.2, 3.2, 4.2), (1.3, 2.3, 3.3, 4.3),
               (1.4, 2.4, 3.4, 4.4), (1.5, 2.5, 3.5, 4.5)],
              dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('t', '<f8')])

and so on for every class.

Or, you can use a single wrapper function to construct all possible combinations of
NumPy type vectors:

In [24]:
# NumPy-like arguments (literally passed through to NumPy)
vector.array(
    [(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
    dtype=[("x", float), ("y", float)],
)

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [25]:
# Pandas-like arguments (dict from names to column arrays)
vector.array({"x": [1.1, 1.2, 1.3, 1.4, 1.5], "y": [2.1, 2.2, 2.3, 2.4, 2.5]})

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [26]:
# As with objects, the coordinate system and dimension is taken from the names of the fields.
vector.array(
    {
        "x": [1.1, 1.2, 1.3, 1.4, 1.5],
        "y": [2.1, 2.2, 2.3, 2.4, 2.5],
        "z": [3.1, 3.2, 3.3, 3.4, 3.5],
        "t": [4.1, 4.2, 4.3, 4.4, 4.5],
    }
)

VectorNumpy4D([(1.1, 2.1, 3.1, 4.1), (1.2, 2.2, 3.2, 4.2), (1.3, 2.3, 3.3, 4.3),
               (1.4, 2.4, 3.4, 4.4), (1.5, 2.5, 3.5, 4.5)],
              dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('t', '<f8')])

In [27]:
vector.array(
    {
        "pt": [1.1, 1.2, 1.3, 1.4, 1.5],
        "phi": [2.1, 2.2, 2.3, 2.4, 2.5],
        "eta": [3.1, 3.2, 3.3, 3.4, 3.5],
        "M": [4.1, 4.2, 4.3, 4.4, 4.5],
    }
)

MomentumNumpy4D([(1.1, 2.1, 3.1, 4.1), (1.2, 2.2, 3.2, 4.2), (1.3, 2.3, 3.3, 4.3),
                 (1.4, 2.4, 3.4, 4.4), (1.5, 2.5, 3.5, 4.5)],
                dtype=[('rho', '<f8'), ('phi', '<f8'), ('eta', '<f8'), ('tau', '<f8')])

Existing NumPy arrays can be viewed as arrays of vectors, but it needs to be a [structured array](https://numpy.org/doc/stable/user/basics.rec.html) with recognized field names.

In [28]:
# NumPy array         # interpret groups of four values as named fields              # give it vector properties and methods
np.arange(0, 24, 0.1).view(
    [("x", float), ("y", float), ("z", float), ("t", float)]
).view(vector.VectorNumpy4D)

VectorNumpy4D([( 0. ,  0.1,  0.2,  0.3), ( 0.4,  0.5,  0.6,  0.7),
               ( 0.8,  0.9,  1. ,  1.1), ( 1.2,  1.3,  1.4,  1.5),
               ( 1.6,  1.7,  1.8,  1.9), ( 2. ,  2.1,  2.2,  2.3),
               ( 2.4,  2.5,  2.6,  2.7), ( 2.8,  2.9,  3. ,  3.1),
               ( 3.2,  3.3,  3.4,  3.5), ( 3.6,  3.7,  3.8,  3.9),
               ( 4. ,  4.1,  4.2,  4.3), ( 4.4,  4.5,  4.6,  4.7),
               ( 4.8,  4.9,  5. ,  5.1), ( 5.2,  5.3,  5.4,  5.5),
               ( 5.6,  5.7,  5.8,  5.9), ( 6. ,  6.1,  6.2,  6.3),
               ( 6.4,  6.5,  6.6,  6.7), ( 6.8,  6.9,  7. ,  7.1),
               ( 7.2,  7.3,  7.4,  7.5), ( 7.6,  7.7,  7.8,  7.9),
               ( 8. ,  8.1,  8.2,  8.3), ( 8.4,  8.5,  8.6,  8.7),
               ( 8.8,  8.9,  9. ,  9.1), ( 9.2,  9.3,  9.4,  9.5),
               ( 9.6,  9.7,  9.8,  9.9), (10. , 10.1, 10.2, 10.3),
               (10.4, 10.5, 10.6, 10.7), (10.8, 10.9, 11. , 11.1),
               (11.2, 11.3, 11.4, 11.5), (11.6, 11.7, 11.8, 11

Since `VectorNumpy2D`, `VectorNumpy3D`, `VectorNumpy4D`, and their momentum equivalents are NumPy array subclasses, all of the normal NumPy methods and functions work on them.

In [29]:
np.arange(0, 24, 0.1).view(
    [("x", float), ("y", float), ("z", float), ("t", float)]
).view(vector.VectorNumpy4D).reshape(6, 5, 2)

VectorNumpy4D([[[( 0. ,  0.1,  0.2,  0.3), ( 0.4,  0.5,  0.6,  0.7)],
                [( 0.8,  0.9,  1. ,  1.1), ( 1.2,  1.3,  1.4,  1.5)],
                [( 1.6,  1.7,  1.8,  1.9), ( 2. ,  2.1,  2.2,  2.3)],
                [( 2.4,  2.5,  2.6,  2.7), ( 2.8,  2.9,  3. ,  3.1)],
                [( 3.2,  3.3,  3.4,  3.5), ( 3.6,  3.7,  3.8,  3.9)]],

               [[( 4. ,  4.1,  4.2,  4.3), ( 4.4,  4.5,  4.6,  4.7)],
                [( 4.8,  4.9,  5. ,  5.1), ( 5.2,  5.3,  5.4,  5.5)],
                [( 5.6,  5.7,  5.8,  5.9), ( 6. ,  6.1,  6.2,  6.3)],
                [( 6.4,  6.5,  6.6,  6.7), ( 6.8,  6.9,  7. ,  7.1)],
                [( 7.2,  7.3,  7.4,  7.5), ( 7.6,  7.7,  7.8,  7.9)]],

               [[( 8. ,  8.1,  8.2,  8.3), ( 8.4,  8.5,  8.6,  8.7)],
                [( 8.8,  8.9,  9. ,  9.1), ( 9.2,  9.3,  9.4,  9.5)],
                [( 9.6,  9.7,  9.8,  9.9), (10. , 10.1, 10.2, 10.3)],
                [(10.4, 10.5, 10.6, 10.7), (10.8, 10.9, 11. , 11.1)],
                

All of the keyword arguments and rules that apply to `vector.obj` construction apply to `vector.array` dtypes.

Geometrical names are used in the dtype, even if momentum-synonyms are used in construction.

In [30]:
vector.array(
    {"px": [1, 2, 3, 4], "py": [1.1, 2.2, 3.3, 4.4], "pz": [0.1, 0.2, 0.3, 0.4]}
)

MomentumNumpy3D([(1., 1.1, 0.1), (2., 2.2, 0.2), (3., 3.3, 0.3), (4., 4.4, 0.4)],
                dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

### Awkward Arrays of vectors

[Awkward Arrays](https://awkward-array.org/) are arrays with more complex data structures than NumPy allows, such as variable-length lists, nested records, missing and even heterogeneous data (multiple data types: use sparingly).

The `vector.Array` function behaves exactly like the [ak.Array](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html) constructor, except that it makes arrays of vectors.

In [31]:
vector.Array(
    [
        [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
        [],
        [{"x": 3, "y": 3.3, "z": 0.3}],
        [
            {"x": 4, "y": 4.4, "z": 0.4},
            {"x": 5, "y": 5.5, "z": 0.5},
            {"x": 6, "y": 6.6, "z": 0.6},
        ],
    ]
)

<VectorArray3D [[{x: 1, y: 1.1, z: 0.1, ... z: 0.6}]] type='4 * var * Vector3D["...'>

If you want _any_ records named "`Vector2D`", "`Vector3D`", "`Vector4D`", "`Momentum2D`", "`Momentum3D`", or "`Momentum4D`" to be interpreted as vectors, register the behaviors globally.

In [32]:
vector.register_awkward()

In [33]:
ak.Array(
    [
        [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
        [],
        [{"x": 3, "y": 3.3, "z": 0.3}],
        [
            {"x": 4, "y": 4.4, "z": 0.4},
            {"x": 5, "y": 5.5, "z": 0.5},
            {"x": 6, "y": 6.6, "z": 0.6},
        ],
    ],
    with_name="Vector3D",
)

<VectorArray3D [[{x: 1, y: 1.1, z: 0.1, ... z: 0.6}]] type='4 * var * Vector3D["...'>

All of the keyword arguments and rules that apply to `vector.obj` construction apply to `vector.Array` field names.

Finally, the `VectorAwkward` mixins can be subclassed to create custom vector classes. The awkward behavior classes and projections must be named as `*Array`. For example, `coffea` uses the following names - `TwoVectorArray`, `ThreeVectorArray`, `PolarTwoVectorArray`, `SphericalThreeVectorArray`, ...

## Vector properties

Any geometrical coordinate can be computed from vectors in any coordinate system; they'll be provided or computed as needed.

In [34]:
vector.obj(x=3, y=4).rho

5.0

In [35]:
vector.obj(rho=5, phi=0.9273).x

2.9999808719721477

In [36]:
vector.obj(rho=5, phi=0.9273).y

4.000014345949428

In [37]:
vector.obj(x=1, y=2, z=3).theta

0.6405223126794245

In [38]:
vector.obj(x=1, y=2, z=3).eta

1.103586841560145

Some properties are not coordinates, but derived from them.

In [39]:
vector.obj(x=1, y=2, z=3).costheta

0.8017837257372732

In [40]:
vector.obj(x=1, y=2, z=3).mag  # spatial magnitude

3.7416573867739413

In [41]:
vector.obj(x=1, y=2, z=3).mag2  # spatial magnitude squared

14

These properties are provided because they can be computed faster or with more numerical stability in different coordinate systems. For instance, the magnitude ignores `phi` in polar coordinates.

In [42]:
vector.obj(rho=3, phi=0.123456789, z=4).mag2

25

Momentum vectors have geometrical properties as well as their momentum-synonyms.

In [43]:
vector.obj(px=3, py=4).rho

5.0

In [44]:
vector.obj(px=3, py=4).pt

5.0

In [45]:
vector.obj(x=1, y=2, z=3, E=4).tau

1.4142135623730951

In [46]:
vector.obj(x=1, y=2, z=3, E=4).mass

1.4142135623730951

Here's the key thing: _arrays of vectors return arrays of coordinates_.

In [47]:
vector.array(
    {
        "x": [1.0, 2.0, 3.0, 4.0, 5.0],
        "y": [1.1, 2.2, 3.3, 4.4, 5.5],
        "z": [0.1, 0.2, 0.3, 0.4, 0.5],
    }
).theta

array([1.50363023, 1.50363023, 1.50363023, 1.50363023, 1.50363023])

In [48]:
vector.Array(
    [
        [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
        [],
        [{"x": 3, "y": 3.3, "z": 0.3}],
        [{"x": 4, "y": 4.4, "z": 0.4}, {"x": 5, "y": 5.5, "z": 0.5}],
    ]
).theta

<Array [[1.5, 1.5], [], [1.5], [1.5, 1.5]] type='4 * var * float64'>

In [49]:
# Make a large, random NumPy array of 3D momentum vectors.
array = (
    np.random.normal(0, 1, 150)
    .view([(x, float) for x in ("x", "y", "z")])
    .view(vector.MomentumNumpy3D)
    .reshape(5, 5, 2)
)
array

MomentumNumpy3D([[[(-1.28111366, -0.03739087, -0.07545124),
                   ( 1.06708043,  0.97346557, -0.19277744)],
                  [(-0.23630574,  0.44316921, -1.22301664),
                   (-0.21826562,  0.31016588, -0.28959995)],
                  [(-1.58731867, -0.16787318,  0.78579993),
                   ( 1.32631401, -0.42905606, -1.10015408)],
                  [(-1.98659247,  0.38280964,  1.58770278),
                   ( 0.74213839,  0.30165301, -0.25113917)],
                  [( 0.78490232,  1.26226008, -0.76560468),
                   ( 0.25209577, -0.68758547,  1.45163241)]],

                 [[(-0.14200396, -0.14399414, -0.35272254),
                   (-0.67984015,  1.30776276,  0.36465356)],
                  [(-0.22503716,  0.63175451, -0.01989071),
                   ( 0.82195282,  0.13522461,  0.09509848)],
                  [( 0.03447106,  0.96871536,  0.21010707),
                   (-0.8698646 , -0.77621204,  0.40058609)],
                  [( 0.2084806

In [50]:
# Get the transverse momentum of each one.
array.pt

array([[[1.28165919, 1.44440156],
        [0.50223436, 0.37926607],
        [1.59617103, 1.39398635],
        [2.02313936, 0.8011017 ],
        [1.48639569, 0.73234286]],

       [[0.20223609, 1.47391522],
        [0.67063811, 0.83300188],
        [0.96932848, 1.16583427],
        [1.03666692, 0.71645964],
        [0.66843387, 3.05177208]],

       [[1.18286423, 1.80961898],
        [1.30590333, 0.48769305],
        [2.52680347, 1.71674138],
        [1.03814121, 2.01079584],
        [1.25899188, 0.96300356]],

       [[1.23175499, 0.94237052],
        [2.41149481, 3.09874755],
        [2.43897206, 1.56034151],
        [0.65498318, 1.40073124],
        [1.60128582, 0.99115771]],

       [[0.91828405, 1.87356156],
        [1.26668754, 3.17403845],
        [1.85695036, 2.39404478],
        [1.02207992, 1.88134866],
        [0.96901934, 2.58520861]]])

In [51]:
# The array and its components have the same shape.
array.shape

(5, 5, 2)

In [52]:
array.pt.shape

(5, 5, 2)

In [53]:
# Make a large, random Awkward Array of 3D momentum vectors.
array = vector.Array(
    [
        [
            {x: np.random.normal(0, 1) for x in ("px", "py", "pz")}
            for inner in range(np.random.poisson(1.5))
        ]
        for outer in range(50)
    ]
)
array

<MomentumArray3D [[], [{x: -0.0764, ... z: 0.0521}]] type='50 * var * Momentum3D...'>

In [54]:
# Get the transverse momentum of each one, in the same nested structure.
array.pt

<Array [[], [0.586, ... 0.433, 2.5, 0.772]] type='50 * var * float64'>

In [55]:
# The array and its components have the same list lengths (and can therefore be used together in subsequent calculations).
ak.num(array)

<Array [0, 2, 2, 5, 2, 2, ... 0, 1, 2, 1, 0, 3] type='50 * int64'>

In [56]:
ak.num(array.pt)

<Array [0, 2, 2, 5, 2, 2, ... 0, 1, 2, 1, 0, 3] type='50 * int64'>

## Vector methods

Vector methods require arguments (in parentheses), which may be scalars or other vectors, depending on the calculation.

In [57]:
vector.obj(x=3, y=4).rotateZ(0.1)

VectorObject2D(x=2.585678829246765, y=4.279516911052588)

In [58]:
vector.obj(rho=5, phi=0.4).rotateZ(0.1)

VectorObject2D(rho=5, phi=0.5)

In [59]:
# Broadcasts a scalar rotation angle of 0.5 to all elements of the NumPy array.
print(
    vector.array({"rho": [1, 2, 3, 4, 5], "phi": [0.1, 0.2, 0.3, 0.4, 0.5]}).rotateZ(
        0.5
    )
)

[(1., 0.6) (2., 0.7) (3., 0.8) (4., 0.9) (5., 1. )]


In [60]:
# Matches each rotation angle to an element of the NumPy array.
print(
    vector.array({"rho": [1, 2, 3, 4, 5], "phi": [0.1, 0.2, 0.3, 0.4, 0.5]}).rotateZ(
        np.array([0.1, 0.2, 0.3, 0.4, 0.5])
    )
)

[(1., 0.2) (2., 0.4) (3., 0.6) (4., 0.8) (5., 1. )]


In [61]:
# Broadcasts a scalar rotation angle of 0.5 to all elements of the Awkward Array.
print(
    vector.Array(
        [[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]
    ).rotateZ(0.5)
)

[[{rho: 1, phi: 0.6}, {rho: 2, phi: 0.7}], [], [{rho: 3, phi: 0.8}]]


In [62]:
# Broadcasts a rotation angle of 0.1 to both elements of the first list, 0.2 to the empty list, and 0.3 to the only element of the last list.
print(
    vector.Array(
        [[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]
    ).rotateZ([0.1, 0.2, 0.3])
)

[[{rho: 1, phi: 0.2}, {rho: 2, phi: 0.3}], [], [{rho: 3, phi: 0.6}]]


In [63]:
# Matches each rotation angle to an element of the Awkward Array.
print(
    vector.Array(
        [[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]
    ).rotateZ([[0.1, 0.2], [], [0.3]])
)

[[{rho: 1, phi: 0.2}, {rho: 2, phi: 0.4}], [], [{rho: 3, phi: 0.6}]]


Some methods are equivalent to binary operators.

In [64]:
vector.obj(x=3, y=4).scale(10)

VectorObject2D(x=30, y=40)

In [65]:
vector.obj(x=3, y=4) * 10

VectorObject2D(x=30, y=40)

In [66]:
10 * vector.obj(x=3, y=4)

VectorObject2D(x=30, y=40)

In [67]:
vector.obj(rho=5, phi=0.5) * 10

VectorObject2D(rho=50, phi=0.5)

Some methods involve more than one vector.

In [68]:
vector.obj(x=1, y=2).add(vector.obj(x=5, y=5))

VectorObject2D(x=6, y=7)

In [69]:
vector.obj(x=1, y=2) + vector.obj(x=5, y=5)

VectorObject2D(x=6, y=7)

In [70]:
vector.obj(x=1, y=2).dot(vector.obj(x=5, y=5))

15

In [71]:
vector.obj(x=1, y=2) @ vector.obj(x=5, y=5)

15

The vectors can use different coordinate systems. Conversions are necessary, but minimized for speed and numeric stability.

In [72]:
vector.obj(x=3, y=4) @ vector.obj(x=6, y=8)  # both are Cartesian, dot product is exact

50

In [73]:
vector.obj(rho=5, phi=0.9273) @ vector.obj(
    x=6, y=8
)  # one is polar, dot product is approximate

49.99999999942831

In [74]:
vector.obj(x=3, y=4) @ vector.obj(
    rho=10, phi=0.9273
)  # one is polar, dot product is approximate

49.99999999942831

In [75]:
vector.obj(rho=5, phi=0.9273) @ vector.obj(
    rho=10, phi=0.9273
)  # both are polar, a formula that depends on phi differences is used

50.0

In Python, some "operators" are actually built-in functions, such as `abs`.

In [76]:
abs(vector.obj(x=3, y=4))

5.0

Note that `abs` returns

   * `rho` for 2D vectors
   * `mag` for 3D vectors
   * `tau` (`mass`) for 4D vectors

Use the named properties when you want magnitude in a specific number of dimensions; use `abs` when you want the magnitude for any number of dimensions.

The vectors can be from different backends. Normal rules for broadcasting Python numbers, NumPy arrays, and Awkward Arrays apply.

In [77]:
vector.array({"x": [1, 2, 3, 4, 5], "y": [0.1, 0.2, 0.3, 0.4, 0.5]}) + vector.obj(
    x=10, y=5
)

VectorNumpy2D([(11., 5.1), (12., 5.2), (13., 5.3), (14., 5.4), (15., 5.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [78]:
(
    vector.Array(
        [  # an Awkward Array of vectors
            [{"x": 1, "y": 1.1}, {"x": 2, "y": 2.2}],
            [],
            [{"x": 3, "y": 3.3}],
            [{"x": 4, "y": 4.4}, {"x": 5, "y": 5.5}],
        ]
    )
    + vector.obj(x=10, y=5)  # and a single vector object
)

<VectorArray2D [[{x: 11, y: 6.1}, ... x: 15, y: 10.5}]] type='4 * var * Vector2D...'>

In [79]:
(
    vector.Array(
        [  # an Awkward Array of vectors
            [{"x": 1, "y": 1.1}, {"x": 2, "y": 2.2}],
            [],
            [{"x": 3, "y": 3.3}],
            [{"x": 4, "y": 4.4}, {"x": 5, "y": 5.5}],
        ]
    )
    + vector.array(
        {"x": [4, 3, 2, 1], "y": [0.1, 0.1, 0.1, 0.1]}
    )  # and a NumPy array of vectors
)

<VectorArray2D [[{x: 5, y: 1.2}, ... x: 6, y: 5.6}]] type='4 * var * Vector2D["x...'>

Some operations are defined for 2D or 3D vectors, but are usable on higher-dimensional vectors because the additional components can be ignored or are passed through unaffected.

In [80]:
vector.obj(rho=1, phi=0.5).deltaphi(
    vector.obj(rho=2, phi=0.3)
)  # deltaphi is a planar operation (defined on the transverse plane)

0.20000000000000018

In [81]:
vector.obj(rho=1, phi=0.5, z=10).deltaphi(
    vector.obj(rho=2, phi=0.3, theta=1.4)
)  # but we can use it on 3D vectors

0.20000000000000018

In [82]:
vector.obj(rho=1, phi=0.5, z=10, t=100).deltaphi(
    vector.obj(rho=2, phi=0.3, theta=1.4, tau=1000)
)  # and 4D vectors

0.20000000000000018

In [83]:
vector.obj(rho=1, phi=0.5).deltaphi(
    vector.obj(rho=2, phi=0.3, theta=1.4, tau=1000)
)  # and mixed dimensionality

0.20000000000000018

This is especially useful for giving 4D vectors all the capabilities of 3D vectors.

In [84]:
vector.obj(x=1, y=2, z=3).rotateX(np.pi / 4)

VectorObject3D(x=1, y=-0.7071067811865472, z=3.5355339059327378)

In [85]:
vector.obj(x=1, y=2, z=3, tau=10).rotateX(np.pi / 4)

VectorObject4D(x=1, y=-0.7071067811865472, z=3.5355339059327378, tau=10)

In [86]:
vector.obj(pt=1, phi=1.3, eta=2).deltaR(vector.obj(pt=2, phi=0.3, eta=1))

1.4142135623730951

In [87]:
vector.obj(pt=1, phi=1.3, eta=2, mass=5).deltaR(
    vector.obj(pt=2, phi=0.3, eta=1, mass=10)
)

1.4142135623730951

For a few operations - `+`, `-`, `==`, `!=`, ... - the dimension of the vectors should be equal. This can be achieved by using the `like` method, `to_{coordinate_name}` methods, `to_Vector*D` methods. The `to_Vector*D` methods provide more flexibility to the users, that is, new coordinate values can be passed into the methods as named arguments.

In [88]:
v1 = vector.obj(x=1, y=2, z=3)
v2 = vector.obj(x=1, y=2)

print(v1 - v2.like(v1))           # transforms v2 to v1's coordinate system (imputes z=0)
print(v1.like(v2) - v2)           # transforms v1 to v2's coordinate system (removes z)
print(v1 - v2.to_xyz())           # transforms v2 to xyz coordinates (imputes z=0)
print(v1.to_xy() - v2)            # transforms v1 to xy coordinates (removes z)
print(v1 - v2.to_Vector3D(z=3))   # transforms v2 to 3D (imputes z=3)
print(v1.to_Vector2D() - v2)      # transforms v1 to 2D (removes z)

VectorObject3D(x=0, y=0, z=3.0)
VectorObject2D(x=0, y=0)
VectorObject3D(x=0, y=0, z=3)
VectorObject2D(x=0, y=0)
VectorObject3D(x=0, y=0, z=0)
VectorObject2D(x=0, y=0)


Similarly, for a few vector methods, the dimension of the input vectors are type checked strictly.

For instance, a cross-product is only defined for 3D and 7D vectors; hence, running the method on a 4D vector will error out.

In [89]:
vector.obj(x=0.1, y=0.2, z=0.3).cross(vector.obj(x=0.4, y=0.5, z=0.6))

VectorObject3D(x=-0.03, y=0.06, z=-0.030000000000000013)

The (current) list of properties and methods is:

**Planar (2D, 3D, 4D):**

   * `x` (`px`)
   * `y` (`py`)
   * `rho` (`pt`): two-dimensional magnitude
   * `rho2` (`pt2`): two-dimensional magnitude squared
   * `phi`
   * `deltaphi(vector)`: difference in `phi` (signed and rectified to $-\pi$ through $\pi$)
   * `rotateZ(angle)`
   * `transform2D(obj)`: the `obj` must supply components through `obj["xx"]`, `obj["xy"]`, `obj["yx"]`, `obj["yy"]`
   * `is_parallel(vector, tolerance=1e-5)`: only true _if they're pointing in the same direction_
   * `is_antiparallel(vector, tolerance=1e-5)`: only true _if they're pointing in opposite directions_
   * `is_perpendicular(vector, tolerance=1e-5)`

**Spatial (3D, 4D):**

   * `z` (`pz`)
   * `theta`
   * `eta`
   * `costheta`
   * `cottheta`
   * `mag` (`p`): three-dimensional magnitude, does not include temporal component
   * `mag2` (`p2`): three-dimensional magnitude squared
   * `cross`: cross-product (strictly 3D)
   * `deltaangle(vector)`: difference in angle (always non-negative)
   * `deltaeta(vector)`: difference in `eta` (signed)
   * `deltaR(vector)`: $\Delta R = \sqrt{\Delta\phi^2 + \Delta\eta^2}$
   * `deltaR2(vector)`: the above, squared
   * `rotateX(angle)`
   * `rotateY(angle)`
   * `rotate_axis(axis, angle)`: the magnitude of `axis` is ignored, but it must be at least 3D
   * `rotate_euler(phi, theta, psi, order="zxz")`: the arguments are in the same order as [ROOT::Math::EulerAngles](https://root.cern.ch/doc/master/classROOT_1_1Math_1_1EulerAngles.html), and `order="zxz"` agrees with ROOT's choice of conventions
   * `rotate_nautical(yaw, pitch, roll)`
   * `rotate_quaternion(u, i, j, k)`: again, the conventions match [ROOT::Math::Quaternion](https://root.cern.ch/doc/master/classROOT_1_1Math_1_1Quaternion.html).
   * `transform3D(obj)`: the `obj` must supply components through `obj["xx"]`, `obj["xy"]`, etc.
   * `is_parallel(vector, tolerance=1e-5)`: only true _if they're pointing in the same direction_
   * `is_antiparallel(vector, tolerance=1e-5)`: only true _if they're pointing in opposite directions_
   * `is_perpendicular(vector, tolerance=1e-5)`

**Lorentz (4D only):**

   * `t` (`E`, `energy`): follows the [ROOT::Math::LorentzVector](https://root.cern/doc/master/LorentzVectorPage.html) behavior of treating spacelike vectors as negative `t` and negative `tau` and truncating wrong-direction timelike vectors
   * `t2` (`E2`, `energy2`)
   * `tau` (`M`, `mass`): see note above
   * `tau2` (`M2`, `mass2`)
   * `beta`: scalar(s) between $0$ (inclusive) and $1$ (exclusive, unless the vector components are infinite)
   * `deltaRapidityPhi`: $\Delta R_{\mbox{rapidity}} = \Delta\phi^2 + \Delta \mbox{rapidity}^2$
   * `deltaRapidityPhi2`: the above, squared
   * `gamma`: scalar(s) between $1$ (inclusive) and $\infty$
   * `rapidity`: scalar(s) between $0$ (inclusive) and $\infty$
   * `boost_p4(four_vector)`: change coordinate system using another 4D vector as the difference
   * `boost_beta(three_vector)`: change coordinate system using a 3D beta vector (all components between $-1$ and $+1$)
   * `boost(vector)`: uses the dimension of the given `vector` to determine behavior
   * `boostX(beta=None, gamma=None)`: supply `beta` xor `gamma`, but not both
   * `boostY(beta=None, gamma=None)`: supply `beta` xor `gamma`, but not both
   * `boostZ(beta=None, gamma=None)`: supply `beta` xor `gamma`, but not both
   * `transform4D(obj)`: the `obj` must supply components through `obj["xx"]`, `obj["xy"]`, etc.
   * `to_beta3()`: turns a `four_vector` (for `boost_p4`) into a `three_vector` (for `boost_beta3`)
   * `is_timelike(tolerance=0)`
   * `is_spacelike(tolerance=0)`
   * `is_lightlike(tolerance=1e-5)`: note the different tolerance

**All numbers of dimensions:**

   * `unit()`: note the parentheses
   * `dot(vector)`: can also use the `@` operator
   * `add(vector)`: can also use the `+` operator
   * `subtract(vector)`: can also use the `-` operator
   * `scale(factor)`: can also use the `*` operator
   * `equal(vector)`: can also use the `==` operator, but consider `isclose` instead
   * `not_equal(vector)`: can also use the `!=` operator, but consider `isclose` instead
   * `sum()`: can also use the `numpy.sum` or `awkward.sum`, only for NumPy and Awkward vectors
   * `count_nonzero()`: can also use `numpy.count_nonzero` or `awkward.count_nonzero`, only for NumPy and Awkward vectors
   * `count()`: can also use `awkward.count`, only for Awkward vectors
   * `isclose(vector, rtol=1e-5, atol=1e-8, equal_nan=False)`: works like [np.isclose](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html); arrays also have an [allclose](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html) method
   * `to_VectorND(coordinates)`/`to_ND(coordinates)`: replace `N` with the required vector dimension
   * `to_{coordinate-names}`: for example - `to_rhophietatau`
   * `like(other)`: projects the vector into the dimensions of `other`, for example - `two_d_vector.like(three_d_vector)`


## Compiling your Python with Numba

[Numba](https://numba.pydata.org/) is a just-in-time (JIT) compiler for a mathematically relevant subset of NumPy and Python. It allows you to write fast code without leaving the Python environment. The drawback of Numba is that it can only compile code blocks involving objects and functions that it recognizes.

The Vector library includes extensions to inform Numba about vector objects, vector NumPy arrays, and vector Awkward Arrays. At the time of writing, the implementation of vector NumPy arrays is incomplete due to [numba/numba#6148](https://github.com/numba/numba/pull/6148).

For instance, consider the following function:

In [90]:
@nb.njit
def compute_mass(v1, v2):
    return (v1 + v2).mass

In [91]:
compute_mass(vector.obj(px=1, py=2, pz=3, E=4), vector.obj(px=-1, py=-2, pz=-3, E=4))

8.0

When the two `MomentumObject4D` objects are passed as arguments, Numba recognizes them and replaces the Python objects with low-level structs. When it compiles the function, it recognizes `+` as the 4D `add` function and recognizes `.mass` as the `tau` component of the result.

Although this demonstrates that Numba can manipulate vector objects, there is no performance advantage (and a likely disadvantage) to compiling a calculation on just a few vectors. The advantage comes when many vectors are involved, in arrays.

In [92]:
# This is still not a large number. You want millions.
array = vector.Array(
    [
        [
            dict(
                {x: np.random.normal(0, 1) for x in ("px", "py", "pz")},
                E=np.random.normal(10, 1),
            )
            for inner in range(np.random.poisson(1.5))
        ]
        for outer in range(50)
    ]
)
array

<MomentumArray4D [[{x: 0.49, y: -2.77, ... t: 9.31}]] type='50 * var * Momentum4...'>

In [93]:
@nb.njit
def compute_masses(array):
    out = np.empty(len(array), np.float64)
    for i, event in enumerate(array):
        total = vector.obj(px=0.0, py=0.0, pz=0.0, E=0.0)
        for vec in event:
            total = total + vec
        out[i] = total.mass
    return out

In [94]:
compute_masses(array)

array([17.8909645 , 10.55040759, 20.07523827,  9.84004207, 20.67409871,
       27.88106879,  0.        , 32.26388208, 28.44712907,  8.82669677,
        9.77241892,  0.        , 18.92545164, 19.36782081, 16.79226804,
       10.62459674, 32.13326598,  9.57807961, 10.77717215,  0.        ,
       21.0389158 , 30.44473778,  0.        , 50.41526533,  0.        ,
        9.24508557, 22.26878408, 29.88592903,  0.        , 18.96505713,
       10.28562132,  0.        , 21.31843914,  9.0798792 , 21.74269618,
       10.64529248, 10.45420065,  7.9109517 ,  0.        ,  9.09929944,
       29.70836013, 48.5004124 , 18.63789529,  9.89926012,  9.19757276,
       31.6617626 ,  0.        ,  0.        ,  0.        ,  9.07660597])

## Talks about vector

- 9th October 2023 - [What’s new with Vector? First major release is out!](https://indi.to/35ym5) - [PyHEP 2023 (virtual)](https://indico.cern.ch/event/1252095/) [🎥](https://www.youtube.com/watch?v=JHEAb2R3xzE&list=PLKZ9c4ONm-VlAorAG8kR09ZqhMfHiH2LJ&index=10)
- 13th September 2022 - [Constructing HEP vectors and analyzing HEP data using Vector](https://indi.to/bPmMc) - [PyHEP 2022 (virtual)](https://indico.cern.ch/event/1150631/) [🎥](https://www.youtube.com/watch?v=4iveMzrbe7s&list=PLKZ9c4ONm-VkohKG-skzEG_gklMaSgaO7&index=15)
- 20th July 2022 - [Analysis Grand Challenge / HEP Scientific Python Ecosystem](https://indico.cern.ch/event/1151329/timetable/#3-analysis-grand-challenge-hep) - [DANCE/CoDaS@Snowmass 2022 computational and data science software training](https://indico.cern.ch/event/1151329/)
- 25th April 2022 - [Foundation libraries (uproot, awkward, hist, mplhep)](https://indico.cern.ch/event/1126109/contributions/4780138/) - [IRIS-HEP AGC Tools 2022 Workshop](https://indico.cern.ch/event/1126109/) [🎥](https://www.youtube.com/watch?v=O9KvsDMKOmY)
- 3rd November 2021 - [Data handling: uproot, awkward & vector](https://indico.cern.ch/event/1076231/contributions/4560398/) - [IRIS-HEP AGC Tools 2021 Workshop](https://indico.cern.ch/event/1076231/) [🎥](https://indico.cern.ch/event/1076231/contributions/4560398/attachments/2338579/4017718/agc_uproot_awk.mp4)


### Status as of November 17, 2023

First major release of vector is out and the package has reached a stable position. The work is spearheaded by bug reports and feature requests created on GitHub. It can only be improved by your feedback!